<a href="https://colab.research.google.com/github/vishnoiprem/honest-colab/blob/main/kafka_stream_data_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ETL using realtime (Kafka + spark )

# Data Engineering Assessment Test:

IMPORTANT:
- Build the system on a cloud environment.


INSTRUCTIONS:

Download the data from https://www.kaggle.com/wordsforthewise/lending-club. Use the file called rejected_2007_to_2018Q4.csv for the following assignment:
1. Using just the data in 2007, we are interested in the Moving Average of the Risk_Score column (50 rows back). Calculate the MA(50) of the Risk_Score and put it in a new column called RiskScoreMA50. Store this in a Data Store of your choice.
2. Starting in year 2008 to 2009, mock up a Kafka Stream of the data coming into the system. Calculate the MA50 for the incoming Risk Score as well. Then store the data in the data store.
3. Starting year 2009 onwards, you get a new requirement to also calculate the MA100 of the Risk Score and store it, but the data schema must also to be trackable. Make a new column on the data store that has a new column called MA100, update the version of this data store, and do version controlling so that the original features can easily be accessed.
4. The team found out that they did not actually want the MA50, but the EMA50 for the purpose of this calculation (want to edit the feature without changing the name). In this case, they do not want a new column for the EMA50, but replace the existing MA50 with new logic. Edit the logic for the feature creation pipeline, and version control the new version of the feature store.


Bonus Points:

1. Submission of a video demo of the system you built.


![image](diagram.png)
# New Section

*italicized text*#1.Setup standalone Spark


https://databricks.com/discover/demos/delta-**lake**

In [15]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [16]:
import os
from os.path import abspath

import site
import time
import requests
import shutil

In [17]:
# update the time to local time
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Singapore /etc/localtime
!date

Sun Jun 12 20:45:22 +08 2022


In [19]:
'''define sparkhome just in case there would be a situation where I need it later on
This method doesnt come with Hadoop binaries which could impair some functions'''
def get_pyspark():
  !pip install pyspark==3.0.3
  PACKAGES_PATH = site.getsitepackages()[0]
  os.environ["SPARK_HOME"] = f"{PACKAGES_PATH}/pyspark"
get_pyspark()
'''PyDrive is a high-level Python wrapper for the Google Drive API. 
It allows you to easily upload, download, and delete files in your Google Drive from a Python script'''
!pip install -U -q PyDrive

#downlad java because some task will requires the Java Virtual Machine (JVM) to run. 

!apt install openjdk-8-jdk-headless -qq
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 87 not upgraded.


In [20]:
!ls -lrt

total 2043692
-rw-r--r-- 1 root root  392582231 Dec 17  2019  accepted_2007_to_2018Q4.csv.gz
-rw-r--r-- 1 root root  255470782 Dec 17  2019  rejected_2007_to_2018Q4.csv.gz
drwxr-xr-x 8 root root       4096 Jan 12 17:07  kafka_2.13-3.1.0
drwxr-xr-x 1 root root       4096 Jun  1 21:50  sample_data
-rw-r--r-- 1 root root         67 Jun 12 16:25  kaggle.json
-rw-r--r-- 1 root root 1356507910 Jun 12 16:26  lending-club.zip
drwxr-xr-x 2 root root       4096 Jun 12 16:26  accepted_2007_to_2018q4.csv
drwxr-xr-x 2 root root       4096 Jun 12 16:26  rejected_2007_to_2018q4.csv
drwxr-xr-x 3 root root       4096 Jun 12 18:00  sample_rejected_2007_to_2018Q4
drwxr-xr-x 2 root root       4096 Jun 12 18:41  spark-warehouse
-rw-r--r-- 1 root root         67 Jun 12 19:45 'kaggle (1).json'
-rw-r--r-- 1 root root   88130011 Jun 12 19:49  kafka_2.13-3.1.0.tgz


In [125]:
#1)Download the data from https://www.kaggle.com/wordsforthewise/lending-club. Use the file called rejected_2007_to_2018Q4.csv for the following assignment:


Kaggle data
# Easiest way to download kaggle data in Google Colab Posted in General 3 years ago 698

# Please follow the steps below to download and use kaggle data within Google Colab:

# Go to your account, Scroll to API section and Click Expire API Token to remove previous tokens

# Click on Create New API Token - It will download kaggle.json file on your machine.

# Go to your Google Colab project file and run the following commands:

# 1) ! pip install -q kaggle

# 2) from google.colab import files

# files.upload()

# Choose the kaggle.json file that you downloaded 3) ! mkdir ~/.kaggle

# ! cp kaggle.json ~/.kaggle/

# Make directory named kaggle and copy kaggle.json file there. 4) ! chmod 600 ~/.kaggle/kaggle.json

# Change the permissions of the file. 5) ! kaggle datasets list

# That's all ! You can check if everything's okay by running this command.
# Download Data ! kaggle competitions download -c 'name-of-competition'

# Use unzip command to unzip the data:

# For example,

# Create a directory named train,

# ! mkdir train

# unzip train data there,

# ! unzip train.zip -d train

In [21]:

!pip install -q kaggle

from google.colab import files

files.upload()

{}

In [22]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d wordsforthewise/lending-club
!unzip lending-club.zip



mkdir: cannot create directory ‘/root/.kaggle’: File exists
lending-club.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  lending-club.zip
replace accepted_2007_to_2018Q4.csv.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: accepted_2007_to_2018Q4.csv.gz  n

replace accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace rejected_2007_to_2018Q4.csv.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


NameError: ignored

In [23]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -xzf kafka_2.13-3.1.0.tgz

In [24]:
!./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.1.0/config/zookeeper.properties
!./kafka_2.13-3.1.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.1.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 20

Waiting for 10 secs until kafka and zookeeper services are up and running


In [26]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic rejected_2007_to_2018
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic rejected2007to2018


Error while executing topic command : Topic 'rejected_2007_to_2018' already exists.
[2022-06-12 12:48:24,791] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'rejected_2007_to_2018' already exists.
 (kafka.admin.TopicCommand$)
Topic: rejected2007to2018	TopicId: fdQyOCm6S6uXpv5egAk8kg	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: rejected2007to2018	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [28]:
!pip install kafka-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#kafka producer to produce data in the topic: rejected_2007_to_2018

2. Starting in year 2008 to 2009, mock up a Kafka Stream of the data coming into the system. Calculate the MA50 for the incoming Risk Score as well. Then store the data in the data store.
rejected_2007_to_2018

In [27]:
from kafka import KafkaAdminClient, KafkaConsumer, KafkaProducer
from kafka.admin import NewTopic
import json
from json import loads
import threading

from csv import DictReader

def kafka_producer_csv():
  # Required setting for Kafka Producer
  bootstrap_servers = ['localhost:9092']
  topicname = 'rejected_2007_to_2018'
  producer = KafkaProducer(bootstrap_servers = bootstrap_servers)
  producer = KafkaProducer()
  # iterate over each line as a ordered dictionary and print only few column by column name
  with open('rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv','r') as read_obj:
      csv_dict_reader = DictReader(read_obj)
      for row in csv_dict_reader:
          ack = producer.send(topicname, json.dumps(row).encode('utf-8'))
          metadata = ack.get()
          # print(metadata.topic, metadata.partition)

threading.Thread(target=kafka_producer_csv).start()

In [29]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic rejected_2007_to_2018


Topic: rejected_2007_to_2018	TopicId: wwzKe7NLQK-hM325UPldQA	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: rejected_2007_to_2018	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [30]:
!./kafka_2.13-3.1.0/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic rejected_2007_to_2018 --from-beginning --max-messages 5


{"Amount Requested": "1000.0", "Application Date": "2007-05-26", "Loan Title": "Wedding Covered but No Honeymoon", "Risk_Score": "693.0", "Debt-To-Income Ratio": "10%", "Zip Code": "481xx", "State": "NM", "Employment Length": "4 years", "Policy Code": "0.0"}
{"Amount Requested": "1000.0", "Application Date": "2007-05-26", "Loan Title": "Consolidating Debt", "Risk_Score": "703.0", "Debt-To-Income Ratio": "10%", "Zip Code": "010xx", "State": "MA", "Employment Length": "< 1 year", "Policy Code": "0.0"}
{"Amount Requested": "11000.0", "Application Date": "2007-05-27", "Loan Title": "Want to consolidate my debt", "Risk_Score": "715.0", "Debt-To-Income Ratio": "10%", "Zip Code": "212xx", "State": "MD", "Employment Length": "1 year", "Policy Code": "0.0"}
{"Amount Requested": "6000.0", "Application Date": "2007-05-27", "Loan Title": "waksman", "Risk_Score": "698.0", "Debt-To-Income Ratio": "38.64%", "Zip Code": "017xx", "State": "MA", "Employment Length": "< 1 year", "Policy Code": "0.0"}
{"A

In [31]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [32]:
!pip3 install findspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
# spark = pyspark.sql.SparkSession.builder.appName("Group7") \
#   .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.3,org.apache.kafka:kafka-clients:2.13-3.1.0") \
#   .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#   .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#   .config('spark.ui.port', '4050')\
#   .getOrCreate()


In [38]:
spark = pyspark.sql.SparkSession.builder.appName("Group7") \
  .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [39]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:3.0.3 pyspark-shell'

In [ ]:
# spark_version='3.0.3'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:{}'.format(spark_version)


In [40]:
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("kafka.security.protocol", "SSL") \
    .option("failOnDataLoss", "false") \
    .option("subscribe", "rejected_2007_to_2018") \
    .option("includeHeaders", "true") \
    .option("startingOffsets", "latest") \
    .option("spark.streaming.kafka.maxRatePerPartition", "2") \
    .load()

AnalysisException: ignored

In [41]:
spark

In [ ]:
# spark is not working so i am try to FLINK

In [42]:

##Speark streaming now working so trying FLINK

!python -m pip install apache-flink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached py4j-0.10.9.3-py2.py3-none-any.whl (198 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9
    Uninstalling py4j-0.10.9:
      Successfully uninstalled py4j-0.10.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyspark 3.0.3 requires py4j==0.10.9, but you have py4j 0.10.9.3 which is incompatible.
delta-spark 1.2.1 requires pyspark<3.3.0,>=3.2.0, but you have pyspark 3.0.3 which is incompatible.


In [43]:
kafka_df = spark.read.csv("rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv",header=True).withColumn("user_Read_time",current_timestamp()) 




```
[link text](https://)# 4.Demo Spark and Delta Lake```

# New Section

In [44]:
kafka_df.show(5)

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+
|Amount Requested|Application Date|          Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|      user_Read_time|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|                 10%|   481xx|   NM|          4 years|        0.0|2022-06-12 12:54:...|
|          1000.0|      2007-05-26|  Consolidating Debt|     703.0|                 10%|   010xx|   MA|         < 1 year|        0.0|2022-06-12 12:54:...|
|         11000.0|      2007-05-27|Want to consolida...|     715.0|                 10%|   212xx|   MD|           1 year|        0.0|2022-06-12 12:54:...|
|          6000.0|      2007-05-27|             waksman|     698.0|   

In [45]:
kafka_df.columns

['Amount Requested',
 'Application Date',
 'Loan Title',
 'Risk_Score',
 'Debt-To-Income Ratio',
 'Zip Code',
 'State',
 'Employment Length',
 'Policy Code',
 'user_Read_time']

In [48]:
from functools import reduce

oldColumns = kafka_df.schema.names
print(oldColumns)
new_columns=['amount_requested','application_date','loan_title','risk_score','debt_to_income_ratio','zip_code','state','employment_length','policy_code','user_read_time']
print(new_columns)





['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length', 'Policy Code', 'user_Read_time']
['amount_requested', 'application_date', 'loan_title', 'risk_score', 'debt_to_income_ratio', 'zip_code', 'state', 'employment_length', 'policy_code', 'user_read_time']


In [61]:
 df = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], new_columns[idx]), range(len(oldColumns)), kafka_df)
 df.printSchema()
 df.show()

root
 |-- amount_requested: string (nullable = true)
 |-- application_date: string (nullable = true)
 |-- loan_title: string (nullable = true)
 |-- risk_score: string (nullable = true)
 |-- debt_to_income_ratio: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- state: string (nullable = true)
 |-- employment_length: string (nullable = true)
 |-- policy_code: string (nullable = true)
 |-- user_read_time: timestamp (nullable = false)

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+
|amount_requested|application_date|          loan_title|risk_score|debt_to_income_ratio|zip_code|state|employment_length|policy_code|      user_read_time|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|           

In [65]:
 df.show(10)

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+
|amount_requested|application_date|          loan_title|risk_score|debt_to_income_ratio|zip_code|state|employment_length|policy_code|      user_read_time|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|                 10%|   481xx|   NM|          4 years|        0.0|2022-06-12 13:01:...|
|          1000.0|      2007-05-26|  Consolidating Debt|     703.0|                 10%|   010xx|   MA|         < 1 year|        0.0|2022-06-12 13:01:...|
|         11000.0|      2007-05-27|Want to consolida...|     715.0|                 10%|   212xx|   MD|           1 year|        0.0|2022-06-12 13:01:...|
|          6000.0|      2007-05-27|             waksman|     698.0|   

In [66]:
df.write.format("delta").mode("overwrite").save("output_rejected_2007_to_2018Q4")


In [ ]:
!ls -al .

total 2043696
drwxr-xr-x 1 root root       4096 Jun 12 17:56 .
drwxr-xr-x 1 root root       4096 Jun 12 16:15 ..
drwxr-xr-x 2 root root       4096 Jun 12 16:26 accepted_2007_to_2018q4.csv
-rw-r--r-- 1 root root  392582231 Dec 17  2019 accepted_2007_to_2018Q4.csv.gz
drwxr-xr-x 4 root root       4096 Jun  1 21:49 .config
drwxr-xr-x 8 root root       4096 Jun 12 16:29 kafka_2.13-3.1.0
-rw-r--r-- 1 root root   88130011 Jun 12 16:29 kafka_2.13-3.1.0.tgz
-rw-r--r-- 1 root root         67 Jun 12 16:25 kaggle.json
-rw-r--r-- 1 root root 1356507910 Jun 12 16:26 lending-club.zip
drwxr-xr-x 2 root root       4096 Jun 12 16:26 rejected_2007_to_2018q4.csv
-rw-r--r-- 1 root root  255470782 Dec 17  2019 rejected_2007_to_2018Q4.csv.gz
drwxr-xr-x 1 root root       4096 Jun  1 21:50 sample_data
drwxr-xr-x 3 root root       4096 Jun 12 18:00 sample_rejected_2007_to_2018Q4


In [67]:
!pip install delta-spark
from delta.tables import *
from delta import *


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyspark-3.2.1-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.0.3
    Uninstalling pyspark-3.0.3:
      Successfully uninstalled pyspark-3.0.3


In [79]:
deltaTable = DeltaTable.forPath(spark, "output_rejected_2007_to_2018Q4")


In [80]:
# library.registerTempTable("library")
# new_info = spark.sql("SELECT *,CAST(rand(10)*id as bigint) count  from library")

df.printSchema()


root
 |-- amount_requested: string (nullable = true)
 |-- application_date: string (nullable = true)
 |-- loan_title: string (nullable = true)
 |-- risk_score: string (nullable = true)
 |-- debt_to_income_ratio: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- state: string (nullable = true)
 |-- employment_length: string (nullable = true)
 |-- policy_code: string (nullable = true)
 |-- user_read_time: timestamp (nullable = false)



In [81]:
df.registerTempTable("output_rejected_2007_to_2018Q4")


#just use moving avg based on year with rows 


In [94]:
   kafka_df_2007_sql="""SELECT 
        amount_requested ,
        application_date ,
        loan_title ,
        risk_score ,   
        debt_to_income_ratio ,
        zip_code ,
        state ,
        employment_length ,            
        policy_code ,
        user_Read_time,
        avg(risk_score) OVER( PARTITION by To_Date( application_date,'yyyy-MM-dd') ORDER BY user_Read_time ROWS BETWEEN 50 PRECEDING AND CURRENT ROW ) as RiskScoreMA50
        FROM output_rejected_2007_to_2018Q4 where application_date between '2007-01-01' AND '2007-12-31'
        """

In [95]:
new_info = spark.sql(kafka_df_2007_sql)


In [96]:
new_info.show()

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+-----------------+
|amount_requested|application_date|          loan_title|risk_score|debt_to_income_ratio|zip_code|state|employment_length|policy_code|      user_Read_time|    RiskScoreMA50|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+-----------------+
|          5000.0|      2007-11-15|               mm5yn|     676.0|              13.12%|   302xx|   GA|         < 1 year|        0.0|2022-06-12 14:01:...|            676.0|
|         10000.0|      2007-11-15|           MONEYMAID|     465.0|              12.84%|   302xx|   GA|         < 1 year|        0.0|2022-06-12 14:01:...|            570.5|
|          3000.0|      2007-11-15|         OPFLEETSIDE|     481.0|              21.92%|   302xx|   GA|          5 years|        0.0|20

In [92]:
# 4a.keep consistency with schema enforcement
# reference ttps://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html

# Goal : Keep consistency Schema enforcement, also known as schema validation, is a safeguard in Delta Lake that ensures data quality by rejecting writes to a table that do not match the table’s schema.

# Business Problem Solve : ensure data schema is consistent even on objects contain on cloud storage

# Rather than automatically adding the new columns, Delta Lake enforces the schema and stops the write from occurring. To help identify which column(s) caused the mismatch, Spark prints out both schemas in the stack trace for comparison.

# The following types of schema changes are eligible for schema evolution during table appends or overwrites: Adding new columns (this is the most common scenario) Changing of data types from NullType -> any other type, or upcasts from ByteType -> ShortType -> IntegerType

In [99]:
new_info.write.format("delta") \
           .mode("append") \
           .save("output_rejected_2007_to_2018Q4")

AnalysisException: ignored

In [100]:
new_info.write.format("delta") \
          .option("mergeSchema", "true")\
           .mode("append") \
           .save("output_rejected_2007_to_2018Q4")

In [101]:
spark.read.format("delta").load("output_rejected_2007_to_2018Q4")

DataFrame[amount_requested: string, application_date: string, loan_title: string, risk_score: string, debt_to_income_ratio: string, zip_code: string, state: string, employment_length: string, policy_code: string, user_read_time: timestamp, RiskScoreMA50: double]

Data management with Delta Log
keep Versioning
https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html

retrieve information on the operations, user, timestamp, and so on for each write to a Delta table by running the history command. The operations are returned in reverse chronological order. By default table history is retained for 30 days.# New Section

In [103]:


spark.read.text("/content/drive/MyDrive/spark_download/output_rejected_2007_to_2018Q4/_delta_log").show()

AnalysisException: ignored

In [104]:
!ls -lrt

total 2043728
-rw-r--r-- 1 root root  392582231 Dec 17  2019  accepted_2007_to_2018Q4.csv.gz
-rw-r--r-- 1 root root  255470782 Dec 17  2019  rejected_2007_to_2018Q4.csv.gz
drwxr-xr-x 8 root root       4096 Jan 12 17:07  kafka_2.13-3.1.0
drwxr-xr-x 1 root root       4096 Jun  1 21:50  sample_data
-rw-r--r-- 1 root root         67 Jun 12 16:25  kaggle.json
-rw-r--r-- 1 root root 1356507910 Jun 12 16:26  lending-club.zip
drwxr-xr-x 2 root root       4096 Jun 12 16:26  accepted_2007_to_2018q4.csv
drwxr-xr-x 2 root root       4096 Jun 12 16:26  rejected_2007_to_2018q4.csv
drwxr-xr-x 2 root root       4096 Jun 12 18:41  spark-warehouse
-rw-r--r-- 1 root root         67 Jun 12 19:45 'kaggle (1).json'
-rw-r--r-- 1 root root   88130011 Jun 12 20:47  kafka_2.13-3.1.0.tgz
drwxr-xr-x 3 root root      36864 Jun 12 22:03  output_rejected_2007_to_2018Q4


In [105]:
cd output_rejected_2007_to_2018Q4

/content/output_rejected_2007_to_2018Q4


In [106]:
ls -lrt

total 188448
-rw-r--r-- 1 root root 14394942 Jun 12 21:01 part-00001-0b8a430d-504e-4ed6-9d85-a296749e4262-c000.snappy.parquet
-rw-r--r-- 1 root root 19463024 Jun 12 21:01 part-00000-52af9203-a751-4ab1-a2bb-3b85d9ce5d07-c000.snappy.parquet
-rw-r--r-- 1 root root 13321720 Jun 12 21:02 part-00002-626ee7b6-4822-4ccf-ba21-488d04d436c1-c000.snappy.parquet
-rw-r--r-- 1 root root 15206689 Jun 12 21:02 part-00003-449b428f-7996-44c9-94d4-df1a1d03fb0e-c000.snappy.parquet
-rw-r--r-- 1 root root 14641948 Jun 12 21:02 part-00004-311eca70-db0f-488e-9537-5396b8c2b30d-c000.snappy.parquet
-rw-r--r-- 1 root root 13654079 Jun 12 21:02 part-00005-7aab53b3-93ca-41c3-a235-4a9989a07f8e-c000.snappy.parquet
-rw-r--r-- 1 root root 14515096 Jun 12 21:03 part-00006-88740ce5-53fa-4483-89bb-8075940a4112-c000.snappy.parquet
-rw-r--r-- 1 root root 13157527 Jun 12 21:03 part-00007-f4f110d3-a055-4db5-ae0f-7a794619d204-c000.snappy.parquet
-rw-r--r-- 1 root root 12797555 Jun 12 21:03 part-00008-f0343b03-168e-4a45-989c-b45

In [110]:
spark.read.text("output_rejected_2007_to_2018Q4/_delta_log").show()


+--------------------+
|               value|
+--------------------+
|{"commitInfo":{"t...|
|{"metaData":{"id"...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
|{"add":{"path":"p...|
+--------------------+
only showing top 20 rows



In [111]:
fullHistoryDF = deltaTable.history().show()


+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2022-06-12 14:03:...|  null|    null|    WRITE|[mode -> Append, ...|null|    null|     null|          0|          null|         true|[numFiles -> 128,...|        null|
|      0|2022-06-12 13:04:...|  null|    null|    WRITE|[mode -> Overwrit...|null|    null|     null|       null|          null|        false|[numFiles -> 14, ...|        null|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--

In [112]:
deltaTable.history().select("version","operation","timestamp").show()

+-------+---------+--------------------+
|version|operation|           timestamp|
+-------+---------+--------------------+
|      1|    WRITE|2022-06-12 14:03:...|
|      0|    WRITE|2022-06-12 13:04:...|
+-------+---------+--------------------+



In [113]:
deltaTable.history().select("version","operation","timestamp").show()

+-------+---------+--------------------+
|version|operation|           timestamp|
+-------+---------+--------------------+
|      1|    WRITE|2022-06-12 14:03:...|
|      0|    WRITE|2022-06-12 13:04:...|
+-------+---------+--------------------+



In [114]:
# ###Time travel to compare data change and rolling back to previous version


In [116]:
   kafka_df_2008_2009_sql="""SELECT 
        amount_requested ,
        application_date ,
        loan_title ,
        risk_score ,   
        debt_to_income_ratio ,
        zip_code ,
        state ,
        employment_length ,            
        policy_code ,
        user_Read_time,
        avg(risk_score) OVER( PARTITION by To_Date( application_date,'yyyy-MM-dd') ORDER BY user_Read_time ROWS BETWEEN 50 PRECEDING AND CURRENT ROW ) as RiskScoreMA50
        FROM output_rejected_2007_to_2018Q4 where application_date between '2008-01-01' AND '2009-12-31'
        """


In [118]:
kafka_df_2008_2009 = spark.sql(kafka_df_2008_2009_sql)


In [119]:
kafka_df_2008_2009.write.format("delta") \
          .option("mergeSchema", "true")\
           .mode("append") \
           .save("output_rejected_2007_to_2018Q4")

In [120]:
deltaTable.history().show(truncate=False)

+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------------------+------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                        |userMetadata|
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------------------+------------+
|2      |2022-06-12 14:11:05.329|null  |null    |WRITE    |[mode -> Append, partitionBy -> []]   |null|null    |null     |1          |null          |true         |[numFiles -> 197, numOutputBytes -> 3338769, numOutputRows -> 82587]    |null        |


In [121]:
spark.read.format("delta").load("output_rejected_2007_to_2018Q4").show(5)

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+-------------+
|amount_requested|application_date|          loan_title|risk_score|debt_to_income_ratio|zip_code|state|employment_length|policy_code|      user_read_time|RiskScoreMA50|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+-------------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|                 10%|   481xx|   NM|          4 years|        0.0|2022-06-12 13:01:...|         null|
|          1000.0|      2007-05-26|  Consolidating Debt|     703.0|                 10%|   010xx|   MA|         < 1 year|        0.0|2022-06-12 13:01:...|         null|
|         11000.0|      2007-05-27|Want to consolida...|     715.0|                 10%|   212xx|   MD|           1 year|        0.0|2022-06-12 13:01:...| 

In [124]:
kafka_df_2008_2009.show()

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+-----------------+
|amount_requested|application_date|          loan_title|risk_score|debt_to_income_ratio|zip_code|state|employment_length|policy_code|      user_Read_time|    RiskScoreMA50|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+--------------------+-----------------+
|         15000.0|      2009-07-25|         Consolidate|     602.0|               9.78%|   940xx|   CA|          6 years|        0.0|2022-06-12 14:12:...|            602.0|
|         25000.0|      2009-07-25|         Consolidate|     747.0|              15.04%|   902xx|   CA|         < 1 year|        0.0|2022-06-12 14:12:...|            674.5|
|          2000.0|      2009-07-25|         Debt Relief|     515.0|              16.83%|   481xx|   MI|         < 1 year|        0.0|20

In [131]:
deltaTable.history().show(truncate=False)

+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-------------------------------------------------------------------------+------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                         |userMetadata|
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-------------------------------------------------------------------------+------------+
|3      |2022-06-12 14:31:26.368|null  |null    |WRITE    |[mode -> Append, partitionBy -> []]   |null|null    |null     |2          |null          |true         |[numFiles -> 200, numOutputBytes -> 274954297, numOutputRows -> 27617821]|null      

3. Starting year 2009 onwards, you get a new requirement to also calculate the MA100 of the Risk Score and store it, but the data schema must also to be trackable. Make a new column on the data store that has a new column called MA100, update the version of this data store, and do version controlling so that the original features can easily be accessed.


In [128]:
   kafka_df_2009_sql="""SELECT 
        amount_requested ,
        application_date ,
        loan_title ,
        risk_score ,   
        debt_to_income_ratio ,
        zip_code ,
        state ,
        employment_length ,            
        policy_code ,
        user_Read_time,
        avg(risk_score) OVER( PARTITION by To_Date( application_date,'yyyy-MM-dd') ORDER BY user_Read_time ROWS BETWEEN 100 PRECEDING AND CURRENT ROW ) as RiskScoreMA100
        FROM output_rejected_2007_to_2018Q4 where application_date > '2009-01-01'
        """


In [129]:
kafka_df_2009 = spark.sql(kafka_df_2009_sql)


In [130]:
kafka_df_2009.write.format("delta") \
          .option("mergeSchema", "true")\
           .mode("append") \
           .save("output_rejected_2007_to_2018Q4")

In [132]:
deltaTable.history().show(truncate=False)

+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-------------------------------------------------------------------------+------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                         |userMetadata|
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-------------------------------------------------------------------------+------------+
|3      |2022-06-12 14:31:26.368|null  |null    |WRITE    |[mode -> Append, partitionBy -> []]   |null|null    |null     |2          |null          |true         |[numFiles -> 200, numOutputBytes -> 274954297, numOutputRows -> 27617821]|null      

In [126]:
#4. The team found out that they did not actually want the MA50, but the EMA50 for the purpose of this calculation (want to edit the feature without changing the name). In this case, they do not want a new column for the EMA50, but replace the existing MA50 with new logic. Edit the logic for the feature creation pipeline, and version control the new version of the feature store.


#4. The team found out that they did not actually want the MA50, but the EMA50 for the purpose of this calculation (want to edit the feature without changing the name). In this case, they do not want a new column for the EMA50, but replace the existing MA50 with new logic. Edit the logic for the feature creation pipeline, and version control the new version of the feature store.
# New Section

# New Section